<a href="https://colab.research.google.com/github/smahesh2694/NEU_XAI/blob/master/severstal_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image
import pickle

import time 

In [0]:
sz = (1600, 256) #size of input images
sz0 = 256
MASKS = '/content/drive/My Drive/Kaggle/severstal-steel-old/train.csv'
IMAGES = '/content/drive/My Drive/Kaggle/severstal-steel-old/images/train_images/'          #test or train

def enc2mask(encs, shape=(1600,256)):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

In [5]:
df_masks = pd.read_csv(MASKS)
split_df = df_masks['ImageId_ClassId'].str.split('_', n=1, expand=True)
df_masks['Image'] = split_df[0]
normal_data = []
multi = []
for image_code in tqdm(df_masks.Image.unique()):
    if df_masks.groupby(['Image']).EncodedPixels.count().loc[image_code] == 0:
        normal_data.append(image_code)
    elif df_masks.groupby(['Image']).EncodedPixels.count().loc[image_code] > 1:
        multi.append(image_code)

100%|██████████| 12568/12568 [04:56<00:00, 42.32it/s]


In [6]:
len(normal_data)

5902

In [7]:
len(multi)

427

In [0]:
df_masks = pd.read_csv(MASKS)
df_masks['id'] = [id[:-2] for id in df_masks.ImageId_ClassId]
df_masks = pd.DataFrame(df_masks.groupby('id')['EncodedPixels'].apply(list))

fnames = os.listdir(IMAGES)
n_crops = 6
offsets = [32 + sz0*i for i in range(n_crops)]

In [9]:
offsets

[32, 288, 544, 800, 1056, 1312]

In [10]:
df_masks

,EncodedPixels
id,
0002cc93b.jpg,[29102 12 29346 24 29602 24 29858 24 30114 24 ...
00031f466.jpg,"[nan, nan, nan, nan]"
000418bfc.jpg,"[nan, nan, nan, nan]"
000789191.jpg,"[nan, nan, nan, nan]"
0007a71bf.jpg,"[nan, nan, 18661 28 18863 82 19091 110 19347 1..."
...,...
fff0295e1.jpg,"[nan, nan, nan, nan]"
fff02e9c5.jpg,"[nan, nan, 207523 3 207777 9 208030 15 208283 ..."
fffe98443.jpg,"[nan, nan, 105929 5 106177 14 106424 24 106672..."


In [11]:
len(fnames)

12568

In [12]:
datadict = dict()
datadict['0'] = []
datadict['1'] = []
datadict['2'] = []
datadict['3'] = []
datadict['4'] = []
f = open('/content/drive/My Drive/Kaggle/severstal-steel-old/black_image.txt', 'w')
count = 0
for fname in tqdm(fnames):
    if '.jpg' not in fname:
        continue
    img0 = Image.open(os.path.join(IMAGES, fname))
    img0 = np.asarray(img0)
    if fname in normal_data:
        normal_data.remove(fname)
        r = np.random.randint(0,n_crops)
        img = img0[:, offsets[r] : offsets[r]+sz0, :]
        if img.mean() < 5 and img.std() < 5:
            f.write(fname+'\n')
            count += 1
            continue
        img = img.reshape([sz0*sz0*3])
        datadict['0'].append(img)
        continue
    elif fname in multi:
        continue
        
    mask0 = enc2mask(df_masks.loc[fname].EncodedPixels)
    
    for i in range(n_crops):
        img = img0[:, offsets[i] : offsets[i]+sz0, :]
        if img.mean() < 5 and img.std() < 5:
            f.write(fname+'\n')
            count += 1
            continue
        img = img.reshape([sz0*sz0*3])
        mask = mask0[:, offsets[i]:offsets[i]+sz0]
        lab = []
        for v in range(1,5):
            lab.append(list(mask.flat).count(v))
        if mask.max() == 0:
            datadict['0'].append(img)
        else:
            datadict[str(np.argmax(lab)+1)].append(img)
f.write(str(count))
f.close() 


#additional
train_dict = dict()
test_dict = dict()
for k,v in datadict.items():
    num_data = len(v)
    global_idx = np.arange(num_data)
    np.random.shuffle(global_idx)
    train_idx = global_idx[:int(num_data*0.7)]
    test_idx = global_idx[int(num_data*0.7):]
    train_dict[k] = np.array(v)[train_idx]
    test_dict[k] = np.array(v)[test_idx]


train = dict()
test = dict()
train['data'] = []
train['label'] = []
test['data'] = []
test['label'] = []

for k,v in train_dict.items():
    for img in v:
        train['data'].append(img)
        train['label'].append(int(k))
train['data'] = np.array(train['data'])
train['label'] = np.array(train['label'])

for k,v in test_dict.items():
    for img in v:
        test['data'].append(img)
        test['label'].append(int(k))
test['data'] = np.array(test['data'])
test['label'] = np.array(test['label'])

# pickling the array
train_file = open('/content/drive/My Drive/Kaggle/severstal-steel-old/new/train_uni', 'wb')
pickle.dump(train, train_file, protocol=pickle.HIGHEST_PROTOCOL)
train_file.close()
test_file = open('/content/drive/My Drive/Kaggle/severstal-steel-old/new/test_uni', 'wb')
pickle.dump(test, test_file, protocol=pickle.HIGHEST_PROTOCOL)
test_file.close()

info = open('/content/drive/My Drive/Kaggle/severstal-steel-old/new/uni_info.txt', 'w')
s = '0: '+str(len(datadict['0']))+'\t1: '+str(len(datadict['1']))+'\t2: '+str(len(datadict['2']))+'\t3: '+str(len(datadict['3']))+'\t4: '+str(len(datadict['4']))
info.write(s)
info.close()

_X = np.array(train['data'])
_Y = np.array(train['label'])

assert _X.shape[0] == _Y.shape[0]
datadict = dict()
for i in range(5):
    datadict[str(i)]=[]

for d, l in tqdm(zip(_X, _Y)):
    datadict[str(l)].append(d)

for k, v in tqdm(datadict.items()):
    fname = '/content/drive/My Drive/Kaggle/severstal-steel-old/new/unilabel_train_'+ k
    f = open(fname, 'wb')
    pickle.dump(np.array(v), f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()

_X = np.array(test['data'])
_Y = np.array(test['label'])

assert _X.shape[0] == _Y.shape[0]
datadict = dict()
for i in range(5):
    datadict[str(i)]=[]

for d, l in tqdm(zip(_X, _Y)):
    datadict[str(l)].append(d)

for k, v in tqdm(datadict.items()):
    fname = '/content/drive/My Drive/Kaggle/severstal-steel-old/new/unilabel_test_'+ k
    f = open(fname, 'wb')
    pickle.dump(np.array(v), f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()

chunk = dict()
chunk['0']=8
chunk['3']=3

for i in range(5):
    fname = '/content/drive/My Drive/Kaggle/severstal-steel-old/new/unilabel_train_'+ str(i)
    f = open(fname, 'rb')
    data = pickle.load(f)
    f.close()
    f = open('/content/drive/My Drive/Kaggle/severstal-steel-old/new/chunked_unilabel_train_'+ str(i), 'wb')
    if i == 1 or i == 2 or i == 4:
        print ('label ', i, 'has of length', data.shape[0])
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()
    else:
        np.random.shuffle(data)
        print ('label ', i, 'has of length', data.shape[0], 'chunk to ',\
               data.shape[0]//chunk[str(i)]) 
        pickle.dump(data[:int(data.shape[0]//chunk[str(i)])], f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()

chunk = dict()
chunk['0']=3

for i in range(5):
    fname = '/content/drive/My Drive/Kaggle/severstal-steel-old/new/unilabel_test_'+ str(i)
    f = open(fname, 'rb')
    data = pickle.load(f)
    f.close()
    f = open('/content/drive/My Drive/Kaggle/severstal-steel-old/new/chunked_unilabel_test_'+ str(i), 'wb')
    if i != 0:
        print ('label ', i, 'has of length', data.shape[0])
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()
    else:
        np.random.shuffle(data)
        print ('label ', i, 'has of length', data.shape[0], 'chunk to ',\
               data.shape[0]//chunk[str(i)])
        pickle.dump(data[:int(data.shape[0]//chunk[str(i)])], f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()



100%|██████████| 12568/12568 [5:08:11<00:00,  1.83s/it]
26839it [00:00, 735536.54it/s]
100%|██████████| 5/5 [01:51<00:00, 25.28s/it]
11505it [00:00, 740648.44it/s]
100%|██████████| 5/5 [01:04<00:00, 14.59s/it]


label  0 has of length 16620 chunk to  2077
label  1 has of length 935
label  2 has of length 147
label  3 has of length 8166 chunk to  2722
label  4 has of length 971
label  0 has of length 7124 chunk to  2374
label  1 has of length 401
label  2 has of length 63
label  3 has of length 3500
label  4 has of length 417
